In [4]:
from datasets import load_dataset
import os
import json 

livecodebench = load_dataset("livecodebench/code_generation_lite", version_tag="release_v5", trust_remote_code=True, split="test")
print(livecodebench)

Dataset({
    features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
    num_rows: 880
})


In [5]:
from datetime import datetime

# deepseek paper uses lcb from 2024-08 to 2025-01
def is_date_in_range(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S")
    # Define the start and end range
    start_date = datetime(2024, 8, 1)  # August 1, 2024
    end_date = datetime(2025, 2, 1)    # February 1, 2025

    # Check if the date is within the range
    return start_date <= date_obj < end_date

In [6]:
dataset = []
for entry in livecodebench:
    tests = entry["public_test_cases"]
    if isinstance(tests, str):
        try:
            tests = json.loads(tests)
        except json.JSONDecodeError as e:
            print(f"code reward Json Error parsing livecodebench: {e}")
            continue 

    if len(tests) == 0:
        continue 
    new_entry = {
        "problem": entry["question_content"],
        "starter_code": entry["starter_code"],
        "tests": tests,
    }
    dataset.append(new_entry)

print(f'Dataset size: {len(dataset)}')

output_dir = os.path.abspath("../../test/code")
output_file = os.path.join(output_dir, "livecodebench.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)


Dataset size: 879
